In [ ]:
import json
from search.dataset_utils import parse_dataset
from datasets import load_dataset, Dataset, DatasetDict

In [ ]:
data = load_dataset("codegenning/taco_with_gpt-4o-mini_gennedsols")
data_fail = load_dataset("codegenning/F_taco-rl_Llama-3.1-8B-I_sols")
problems_llama = parse_dataset("codegenning/F_taco-rl_Llama-3.1-8B-I_sols", "train")
problems_lcb = parse_dataset("codegenning/F_livecodebench_lite_v2", "test")

In [ ]:
dd = DatasetDict({"train": data_fail["train"].map(lambda row, idx: {"question": row["question"].split("Starter Code")[0].strip()}, with_indices=True)})
# dd.push_to_hub("codegenning/F_taco-rl-Llama-3.1-8B-I_sols_fix", private=True)

In [ ]:
question_to_problem = {}
for i, row in enumerate(dd["train"]):
    assert row["question"] not in question_to_problem
    question_to_problem[row["question"]] = i

In [ ]:
gpt_to_llama_idx = []
fill_llama_sols = [[] for _ in range(len(data_fail["train"]))]
for i, row in list(enumerate(data["train"])):
    if row["problem_str"] not in question_to_problem:
        continue
    gpt_to_llama_idx.append(question_to_problem[row["problem_str"]])
    fill_llama_sols[question_to_problem[row["problem_str"]]] = row["generated_solutions"]

In [ ]:
ndd = DatasetDict({"train": dd["train"].map(lambda row, idx: {"solutions": fill_llama_sols[idx]}, with_indices=True)})

In [ ]:
nndd = ndd["train"].train_test_split(0.01)

In [ ]:
nndd.push_to_hub("codegenning/F_taco-rl_gpt-4omi_ll318bi", private=True)